In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np
import os
import torch
#import torchvision
from torch.utils.data import DataLoader
#from torchvision import transforms
from torch.utils.data import Dataset,SubsetRandomSampler,Sampler
import matplotlib.pyplot as plt
from torch import nn
import torch.nn.functional as F
from torch import optim
import glob
from skimage import io, transform
from PIL import Image
import random
import PIL.ImageEnhance as ie
import copy
from torch.autograd import Variable
import PIL.Image as im
from math import floor
from torchvision.transforms import *
import time

In [3]:
%cd ../content/drive/My Drive

[Errno 2] No such file or directory: '../content/drive/My Drive'
/content


In [0]:
root = "./plant-pathology-2020-fgvc7/images"

In [5]:
print(root)

./plant-pathology-2020-fgvc7/images


In [6]:
df = pd.read_csv("./plant-pathology-2020-fgvc7/my_csv.csv")
csv_file=df.to_numpy()

FileNotFoundError: ignored

In [0]:
for i in range(len(csv_file[:,1])):
  if(csv_file[i,1]=='healthy'):
    csv_file[i,1]=0
  elif(csv_file[i,1]=="multiple_diseases"):
    csv_file[i,1]=1
  elif(csv_file[i,1]=="rust"):
    csv_file[i,1]=2
  elif(csv_file[i,1]=="scab"):
    csv_file[i,1]=3

In [0]:
class PlantDataset(Dataset):
    def __init__(self, root,csv_file, transform=None):
        self.root_dir = root
        self.transform = transform
        self.csv_file = csv_file
        

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.csv_file[idx, 0])
        image = Image.open(img_name).convert('RGB')
        label = self.csv_file[idx, 1]

        if self.transform:
            image = self.transform(image)
        return image,label

In [0]:
transform=transforms.Compose([CenterCrop(512),RandomGrayscale(p=0.5),RandomHorizontalFlip(0.5),RandomRotation(180),RandomVerticalFlip(p=0.5),
                              ToTensor(),Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
                                           ])

In [0]:
trainset = PlantDataset(root=root,csv_file =csv_file , transform=transform)
valset   = PlantDataset(root=root,csv_file =csv_file, transform=transform)

In [16]:
for i in trainset:
  print(i[1])
  break

1


In [0]:
def train_valid_split(dataset, test_size = 0.20, shuffle = False, random_seed = 0):
    length = dataset.__len__()
    indices = list(range(1,length))
    
    if shuffle == True:
        random.seed(random_seed)
        random.shuffle(indices)
    
    if type(test_size) is float:
        split = floor(test_size * length)
    elif type(test_size) is int:
        split = test_size
    else:
        raise ValueError('%s should be an int or a float' % str)
    return indices[split:], indices[:split]

In [0]:
train_idx, valid_idx = train_valid_split(trainset, 0.20)
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [0]:
train_loader = DataLoader(dataset=trainset, sampler = train_sampler,batch_size =12)
valid_loader = DataLoader(dataset=valset, sampler = valid_sampler,batch_size =12)

In [13]:
import torch
model = torch.hub.load('pytorch/vision:v0.5.0', 'resnext50_32x4d', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.5.0


In [0]:
model.fc = nn.Sequential(nn.Linear(in_features=2048,out_features=320),nn.Dropout(p=0.3),nn.Linear(in_features = 320,out_features=4))

In [15]:
model.cuda()

RuntimeError: ignored

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [0]:
model.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [0]:
def test_accuracy(model):
    valid_losses=[]
    correct =0
    total =0
    wrong=0
    model.eval()
    with torch.no_grad():
      for inputs, labels in valid_loader:
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        output = model(inputs)
        loss = criterion(output,labels)
        valid_losses.append(loss.item())
        _, preds = torch.max(output.data, 1)
        correct += torch.sum(preds == labels)
        wrong += torch.sum(preds != labels)
        
      acc = (correct.float()) / ((correct+wrong).float())  
      valid_loss = np.average(valid_losses)
      print(valid_loss)
      return((correct*100)/(correct+wrong))

In [0]:
acc = test_accuracy(model)
print(acc)

0.5084874079279278
tensor(84, device='cuda:0')


/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


In [0]:
import torchvision.models.quantization as models
model_fe = models.resnext101_32x8d(pretrained=True, progress=True, quantize=True)

In [0]:
num_ftrs = model_fe.fc.in_features
from torch import nn

def create_combined_model(model_fe):
  # Step 1. Isolate the feature extractor.
  model_fe_features = nn.Sequential(
    model_fe.quant,  # Quantize the input
    model_fe.conv1,
    model_fe.bn1,
    model_fe.relu,
    model_fe.maxpool,
    model_fe.layer1,
    model_fe.layer2,
    model_fe.layer3,
    model_fe.layer4,
    model_fe.avgpool,
    model_fe.dequant,  # Dequantize the output
  )

  # Step 2. Create a new "head"
  new_head = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(num_ftrs, 4),
  )

  # Step 3. Combine, and don't forget the quant stubs.
  new_model = nn.Sequential(
    model_fe_features,
    nn.Flatten(1),
    new_head,
  )
  return new_model

In [26]:
import torch.optim as optim
new_model = create_combined_model(model_fe)
new_model = new_model.to('cpu')

criterion = nn.CrossEntropyLoss()

# Note that we are only training the head.
optimizer_ft = optim.SGD(new_model.parameters(), lr=0.01, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

NameError: ignored

In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, device='cpu'):
  """
  Support function for model training.

  Args:
    model: Model to be trained
    criterion: Optimization criterion (loss)
    optimizer: Optimizer to use for training
    scheduler: Instance of ``torch.optim.lr_scheduler``
    num_epochs: Number of epochs
    device: Device to run the training on. Must be 'cpu' or 'cuda'
  """
  since = time.time()

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  for epoch in range(num_epochs): 
    model.train()
    valid_losses=[]
    train_losses=[]
    
    for i,(inputs, labels) in enumerate(train_loader):
        
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        
        outputs = model(inputs)
        print(outputs.shape,labels.shape)
        break
        loss = criterion(outputs,labels)
        train_losses.append(loss.item())
        loss.backward()

        optimizer.step()
        scheduler.step()

    correct =0
    total =0
    wrong=0
    model.eval()
    for inputs, labels in valid_loader:
      inputs = inputs.to(device)
      labels = labels.to(device)
      
      output = model(inputs)
      loss = criterion(output,labels)
      valid_losses.append(loss.item())
      _, preds = torch.max(output.data, 1)
      correct += torch.sum(preds == labels)
      wrong += torch.sum(preds != labels)
      
    acc = (correct.float()) / ((correct+wrong).float()) 
    print(epoch , acc) 
    if(acc > best_acc):
      best_acc = acc
      best_model_wts = copy.deepcopy(model.state_dict())

  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))

  # load best model weights
  model.load_state_dict(best_model_wts)
  return model

In [0]:
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
new_model = train_model(model, criterion, optimizer, exp_lr_scheduler,
                        num_epochs=10, device='cpu')


torch.Size([16, 4]) torch.Size([16])
